<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/assignments/assignment_yourname_class8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/academics/programs/index.html)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 8 Assignment: Feature Engineering**

**Student Name: Your Name**

# Assignment Instructions

This assignment is similar to assignment 5, except that you must use feature engineering to solve it.  I provide you with a dataset that contains dimensions and the quality of items of specific shapes.  With the values of 'height', 'width', 'depth'. 'shape', and 'quality' you should try to predict the cost of these items.  You should be able to match very close to solution file, if you feature engineer correctly.  To get full credit your average cost should not be more than 50 off from the solution.  The autocorrector will let you know if you are in this range.

You can find all of the needed CSV files here:

* [Shapes - Training](https://data.heatonresearch.com/data/t81-558/datasets/shapes-train.csv)
* [Shapes - Submit](https://data.heatonresearch.com/data/t81-558/datasets/shapes-test.csv)

Use the training file to train your neural network and submit results for for the data contained in the test/submit file.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems.

**It is unlikely that should need to modify this function.**

In [ ]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# course - The course that you are in, currently t81-558 or t81-559.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,course,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/wu/submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'course':course, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #8 Sample Code

The following code provides a starting point for this assignment.

In [ ]:
!pip install torch

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingRegressor

# This is your student key that I emailed to you at the beginnning of the semester.
# key = "Gx5en9cEVvcsjuijujijisdsdZO4PsI32sgldAXj"  # This is an example key and will not work.

key = ""

# You must also identify your source file.  (modify for your local setup)
# file='/content/drive/My Drive/Colab Notebooks/assignment_solution_class8.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class8.ipynb'  # Windows
# file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class8.ipynb'  # Mac/Linux

file= '/content/drive/My Drive/Colab Notebooks/assignment_ZhijiangLi_class8.ipynb'

# Begin assignment
df_train = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/shapes-train.csv")
df_submit = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/shapes-test.csv")

###
# def calculate_volume(row):
#     if row['shape'] == 'box':
#         return row['height'] * row['width'] * row['depth']
#     elif row['shape'] == 'cylinder':
#         return np.pi * (row['width'] / 2) ** 2 * row['height']
#     elif row['shape'] == 'ellipsoid':
#         return (4/3) * np.pi * (row['height'] / 2) * (row['width'] / 2) * (row['depth'] / 2)
#     return 0

# df_train['volume'] = df_train.apply(calculate_volume, axis=1)
# df_submit['volume'] = df_submit.apply(calculate_volume, axis=1)

# X = df_train[['height', 'width', 'depth', 'shape', 'quality', 'volume']]
# y = df_train['cost']

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('shape', OneHotEncoder(), ['shape']),
#         ('scaler', StandardScaler(), ['height', 'width', 'depth', 'quality', 'volume'])
#     ])


# ## X = preprocessor.fit_transform(X) ## 1
# pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('poly', PolynomialFeatures(degree=2, include_bias=False)),
#     ('regressor', GradientBoostingRegressor(n_estimators=200, max_depth=6, random_state=42))
# ])


# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# pipeline.fit(X_train, y_train)

# # Evaluate on validation set
# y_pred = pipeline.predict(X_val)
# mae = mean_absolute_error(y_val, y_pred)

# X_submit = df_submit[['height', 'width', 'depth', 'shape', 'quality', 'volume']]
# X_submit_transformed = pipeline.predict(X_submit)

# df_submit = pd.DataFrame({
#     'cost': X_submit_transformed.flatten(),  # Match 'cost' header
#     'id': df_submit['id']
# })
###

###
# X_train_tensor = torch.tensor(X_train.astype(np.float32))
# X_val_tensor = torch.tensor(X_val.astype(np.float32))
# y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).view(-1, 1)
# y_val_tensor = torch.tensor(y_val.values.astype(np.float32)).view(-1, 1)


# class CostPredictorNN(nn.Module):
#     def __init__(self):
#         super(CostPredictorNN, self).__init__()
#         self.fc1 = nn.Linear(X_train.shape[1], 64)
#         self.fc2 = nn.Linear(64, 32)
#         self.fc3 = nn.Linear(32, 1)
#         self.relu = nn.ReLU()

#     def forward(self, x):
#         x = self.relu(self.fc1(x))
#         x = self.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


# model = CostPredictorNN()
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)


# epochs = 100
# for epoch in range(epochs):
#     model.train()
#     optimizer.zero_grad()
#     output = model(X_train_tensor)
#     loss = criterion(output, y_train_tensor)
#     loss.backward()
#     optimizer.step()

#     if (epoch+1) % 10 == 0:
#         model.eval()
#         val_output = model(X_val_tensor)
#         val_loss = criterion(val_output, y_val_tensor)
#         # print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')


# model.eval()
# val_predictions = model(X_val_tensor).detach().numpy()
# mae = mean_absolute_error(y_val, val_predictions)
# # print("Mean Absolute Error on Validation Set:", mae)

# X_submit = preprocessor.transform(df_submit[['height', 'width', 'depth', 'shape', 'quality', 'volume']])
# X_submit_tensor = torch.tensor(X_submit.astype(np.float32))
# # submit_predictions = model(X_submit_tensor).detach().numpy() ## 1

# df_submit = pd.DataFrame({'cost': submit_predictions.flatten(), 'id': df_submit['id']})
# df_submit
###

def calculate_volume(row):
    if row['shape'] == 'box':
        return row['height'] * row['width'] * row['depth']
    elif row['shape'] == 'cylinder':
        return np.pi * (row['width'] / 2) ** 2 * row['height']
    elif row['shape'] == 'ellipsoid':
        return (4/3) * np.pi * (row['height'] / 2) * (row['width'] / 2) * (row['depth'] / 2)
    return 0

df_train['volume'] = df_train.apply(calculate_volume, axis=1)
df_submit['volume'] = df_submit.apply(calculate_volume, axis=1)

X = df_train[['height', 'width', 'depth', 'shape', 'quality', 'volume']]
y = df_train['cost']

preprocessor = ColumnTransformer(
    transformers=[
        ('shape', OneHotEncoder(), ['shape']),
        ('scaler', StandardScaler(), ['height', 'width', 'depth', 'quality', 'volume'])
    ])

#
gb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    # ('poly', PolynomialFeatures(degree=2, include_bias=False)),# no this even better
    ('regressor', GradientBoostingRegressor(n_estimators=40, max_depth=6, random_state=42))
    # 200, 6
])


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## testing
gb_pipeline.fit(X_train, y_train)
y_pred_gb = gb_pipeline.predict(X_val)

mae_gb = mean_absolute_error(y_val, y_pred_gb)
# print("Gradient Boosting MAE on Validation Set:", mae_gb)

# Model
class CostPredictorNN(nn.Module):
    def __init__(self, input_size):
        super(CostPredictorNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

X_train_tensor = torch.tensor(preprocessor.fit_transform(X_train).astype(np.float32))
X_val_tensor = torch.tensor(preprocessor.transform(X_val).astype(np.float32))
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).view(-1, 1)
y_val_tensor = torch.tensor(y_val.values.astype(np.float32)).view(-1, 1)

nn_model = CostPredictorNN(X_train_tensor.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(nn_model.parameters(), lr=0.001)

# Train
epochs = 100
for epoch in range(epochs):
    nn_model.train()
    optimizer.zero_grad()
    output = nn_model(X_train_tensor)
    loss = criterion(output, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        nn_model.eval()
        val_output = nn_model(X_val_tensor)
        val_loss = criterion(val_output, y_val_tensor)
        # print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

nn_model.eval()
val_predictions_nn = nn_model(X_val_tensor).detach().numpy()
mae_nn = mean_absolute_error(y_val, val_predictions_nn)
# print("MAE on Validation Set:", mae_nn)

X_submit = df_submit[['height', 'width', 'depth', 'shape', 'quality', 'volume']]

if mae_gb < mae_nn:
    X_submit_transformed = gb_pipeline.predict(X_submit)
else:
    X_submit_tensor = torch.tensor(preprocessor.transform(X_submit).astype(np.float32))
    X_submit_transformed = nn_model(X_submit_tensor).detach().numpy().flatten()




df_submit = pd.DataFrame({
    'cost': X_submit_transformed.flatten(),
    'id': df_submit['id']
})


#
submit(source_file=file,data=[df_submit],key=key,course='t81-558',no=8)

Success: Submitted Assignment 8 (t81-558) for l.zhijiang:
You have submitted this assignment 28 times. (this is fine)
Note: The mean difference 1.1142096244070672 for column 'cost' is acceptable and is less than the maximum allowed value of '50.0' for this assignment.
No warnings or errors (only notes), you will probably do well, but no guarantee. :-)
